<br>
<img src="img\imdb.png" >
<br>

Utilize o dataset train.csv para os exercicios.

# Exercício: 

Estamos em 2015 e você trabalha em uma empresa que compra filmes para passar no cinema. Bons filmes custam caro, assim como filmes ruins barato, mas Bons filmes atraem mais público. Seu chefe sabe que você está aprendendo Machine Learning e te propõe a seguinte tarefa:  - Precisamos achar uma boa oportunidade que nos faça aumentar o lucro da empresa!

Você conseguirá aumentar o lucro da empresa quando comprar um filme que é subvalorizado mas que as features dele indicam que será um bom filme.

Se pudessemos descobrir quais filmes que as pessoas acham que seriam ruins mas na verdade são bons, poderiamos investir apenas nestes e ganhar a diferença já que foi mal precificado.


Vamos partir dos seguintes pressupostos:
- Um filme bom ou ruim do ponto de vista de quem está precificando será o tamanho do seu orçamento (budget).

- Um filme Bom/Ruim do ponto de vista do público serão filmes com altas notas (imdb_score).

Então vamos procurar os filmes que tenham uma boa relação entre budget e score.
(é como se quisessemos comprar scores pois isso que o público quer ver mas os donos dos filmes vendem baseado no budget do filme).

O primeiro passo é entendermos bem o problema e os dados disponíveis.

Por exemplo, se vamos comprar um filme, algumas features não estarão disponiveis antes da compra (elas são criadas apenas após a compra) para predizer o rating como as vendas de ingresso, faturamento, etc.

Depois separamos nos nossos dados quais serão nossas variáveis explicativas (Os X) e qual é nossa variável target (nosso Y).

Caso algumas das nossas variáveis explicativas sejam categórias, precisamos transforma-las em numéricas para que o modelo consiga entende-la.

Podemos utilizar uma função do pandas dessa forma:

``` df = pd.get_dummies(df) ```

O último passo antes de rodar um modelo é dividir nossos dados em duas partes, uma para treinar o modelo e outra para testa-lo, podemos fazer uma amostra aleatória ou usar a função do sklearn que nos ajuda com isso:

```
from sklearn.model_selection import train_test_split

 X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.33, random_state=42)
```
Para isso Treine um Modelo de Machine Learning supervisionado que aprenda a predizer as notas dos filmes.

Utilize os 4 passos visto na primeira aula de machine learning:
```from sklearn.linear_model import LinearRegression```

Após treinar o modelo vamos ver como estão as [métricas](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) de assertividade.

Analisar o R2, RMSE, MSE. Qual é o melhor nesse caso?

Exemplo do MSE
```
from sklearn.metrics import mean_squared_error

mean_squared_error(y_true, y_pred)
```

Como poderiamos melhorar esse modelo? Será que existe algum método para colocar todas as variáveis ao quadrado, cubo, etc para que possamos capturar efeitos não lineares? Óbviamente existe e já foi implementado no sklearn, podemos usar a função:

```
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3)

df = poly.fit_transform(df)
```

Aparentemente sempre que aumentamos o grau do polinomio nosso modelo fica melhor, porque não podemos botar um número alto como 10?

Rode o algoritmo com um grau = 10 e compare as métricas de treino contra as métricas de teste. O que está acontecendo?

Parece que estamos chegando perto de um modelo bom para nosso objetivo de identificar oportunidade nos filmes. Será que poderiamos utilizar outro método de regressão mais robusto? Vamos tentar utilizar um dos melhores métodos que utiliza arvores aleatórias (usado geralmente em classificação e as estudaremos mais profundamente no módulo 4) como modelo:

Treine agora o mesmo modelo utilizando um modelo não-linear (não exige que as correlações sejam lineares) e compare as métricas.

```from sklearn.ensemble import RandomForestRegressor```



Agora estamos em 2016. Use seu modelo treinado com todos os filmes até 2015 para responder a pergunta do seu chefe com os filmes que apareceram em 2016: Quais são os filmes que são realmente oportunidades de ganhar dinheiro (altos ratings, baixo custo).

# Passo 1 - Entender o problema

O problema está descrito no tutorial acima. O que não está escrito é qual abordagem deveriamos ter para resolve-lo.

Vamos pensar em uma possibilidade de solução aqui e escreveremos todo o restante do código pensando em como alcançar esse resultado.

Não existe um só caminho para resolver esse problema, mas a solução que eu pensei em aplicar é a seguinte:

    1) de um lado os donos dos filmes precificam seus produtos com base em quanto gastaram.

    2) Seguindo o pressuposto do problema, o sucesso de bilheteria do filme depende apenas das notas que as pessoas vão dar a ele.

Com isso chegamos a conclusão que o melhor filme para apostarmos são aqueles que a relação entre o preço (1-custo) e o (2-receita) é a mais vantajoso para nós.

Uma forma de pensar é que temos que "comprar" scores então procuraremos os scores mais baratos que será dado por (1)/(2) -> custo do filme / beneficio do filme.

Então precisamos treinar um modelo de machine learning que quando chegar um filme novo, prediga com o máximo de acurácia a nota média que as pessoas darão para ele ( o quociente na nossa equação) já que o númerador (o custo do filme) já saberemos.

In [ ]:
# improtando as bibliotecas
import pandas as pd
import numpy as np

In [ ]:
# lendo os dados e armazenando em uma variável
df_raw = pd.read_csv("../../99 Datasets/imdb_train.zip")

## Criando uma cópia para o DataFrame

Agora vamos fazer uma cópia do nosso dataframe; note que usamos a função deep=True pois só assim garantimos que o dado está duplicado e não é apenas um "atalho" para o dataset original

In [ ]:
# agora vamos fazer uma cópia do nosso dataframe.
df = df_raw.copy(deep=True)

In [ ]:
# verificando o tamanho dos dados
df.shape

# Passo 2 - Escolher as variáveis.


Definido claramente o que vamos fazer, vamos olhar as variáveis disponiveis e ver o que faz ou não sentido incluir.

Diferentemente de modelagem estatistica, em machine learning não estamos preocupados com as variáveis serem correlacionadas, então não há um bom motivo (por hora) para retirarmos variáveis do modelo, portanto, meu primeiro modelo de 'benchmark' terá o máximo de variáveis/features possível.

Algumas são facilmente excluidas, pois mesmo que sejam boas preditoras como o tamanho da bilheteria, ela só estará disponível após o filme ser lançado. Parte bastante importante da definição de um modelo de machine learning é pensar nesses pontos, de quais variáveis estarão disponiveis em um modelo de produção e isso, muitas vezes, pode ser bastante complicado por conta do carater temporal das variáveis tornando-se um "leakage" ou seja, um vazamento de dados.


Variáveis como gross sabemos que não estará disponivel antes do lançamento então não vamos usa-la para treinar nosso modelo.

Outra variável que não faz sentido para um modelo de machine é aquela que identifica o filme como um id, index ou o nome do filme.

In [ ]:
# listando as colunas do DataFrame
list(df_raw)

In [ ]:
# escolhendo colunas para dropar
columns_to_drop = ['Unnamed: 0', 'gross', 'movie_title', 'plot_keywords']

In [ ]:
# definindo um novo DataFrame com base no original
df = df_raw.drop(columns_to_drop, axis=1)

In [ ]:
# listando as colunas do novo DataFrame
list(df)

# Passo 3 - Tratando as variáveis.

**Essa é a etapa mais longa, mais importante e mais dificil de um problema de machine learning.** (falaremos mais sobre essas técnicas nas próximas aulas)

Essa etapa podemos usar todos os tratamentos de variáveis que aprendemos na modelagem estatistica quando estavamos rodando regressões simples. Não vou repetir os tratamentos mas todos podem ser utilizados.


## Campos nulos

Quando rodamos um dff.describe() percebemos muitos campos nulos e há diversas formas de trata-los.

Alternativas para tratar nulos.

1) Deleter todos os nulos.

```
df = df.dropna()
```

2) Substituir os nans pelas médias (nas numéricas) ou modas (nas categóricas).

df['numericas'] = df['numericas'].fillna(df.mean())


3) rodar um "pré-ML" que prevê os valores faltantes.



4) Usar um algoritmo robusto para campos nulos.

ex:

```
import xgboost as xgb
```

Vamos usar uma abordagem mista. Nas variáveis numéricas vamos substituir pela média e nas variáveis categóricas vamos excluir as linhas com campos nulos para aprender as duas metodologias. 

(conforme vimos em estatistica, excluir linhas que não são aleatórias enviesam nosso modelo e não deve ser feito a menos que você tenha certeza que aqueles nulos são por alguma falha aleatória o que é bastante raro. Não sendo aleatório devemos tratar com uma das outras metodologias, principalmente a número 4).

In [ ]:
# verificando os tipos das variáveis
df.dtypes

In [ ]:
# Definindo um dataset apenas com colunas numéricas
df_numeric = df.select_dtypes(include=[np.number]) 
numericas = list(df_numeric)

Nessa função selecionamos todas as colunas númericas  
- substituimos seus valores pelas médias das colunas
- A função fillna retorna todos os campos nulos 
- função df.mean retorna as médias de cada coluna

In [ ]:
# selecionando as colunas numéricas e preenchendo com a média
df[numericas] = df[numericas].fillna(df.mean())

Já nas variáveis categóricas (que contém texto ao invés de números), vamos substituir os campos nulos por uma nova string que representa que aquele campo é faltante.



In [ ]:
# substitui os dados faltantes por 'na'
df = df.fillna('na') 
# se sobrar alguma linha (nao deveria), dropamos essas linhas.
df = df.dropna() 
# verificando o tamanho do DataFrame
df.shape

## Criar variáveis dummies para features categóricas.

Basicamente os algoritmos de machine learning não conseguem interpretar textos como o genêro de um filme ou os atores que participaram então precisamos transformar essas colunas categóricas em colunas numéricas para que o algoritmo possa capturar seus efeitos e predizer nosso target.

Substituir as categorias por números dessa forma: COluna genêro de filme com valores [Terror=1, Comédia=2, Ação=3] embora faça o algoritmo rodar, não funcionará bem como variável explicativa pois elas não representam uma sequencia numérica. A forma mais simples de fazermos isso virar uma variável explicativa é pegar cada uma das categorias possíveis e transformar em uma coluna forma única que recebe apenas dois valores, 0 e 1. 0 se o valor não estiver presente e 1 caso esteja presente. Dessa forma nossa  representação ficará da seguinte forma. 

- Coluna 1 = genero_terror
- Coluna 2 = genero_comédia
- Coluna 3 = genero_ação

Assumindo 1 ou 0 dependendo do filme.

Uma observação sobre essa técnica é que não é necessário colocar todas as possibilidades, uma delas se torna redundante e ao modelar esse caso podemos excluir uma delas.

Essa técninca chamamos de one_hot (sklearn) e no pandas temos a função get_dummies que faz o mesmo processo.

Outra etapa de tratamento das variáveis consiste em transformar as variáveis em funções lineares (se estivermos rodando modelos lineares) através da aplicação de log nas variáveis "explosivas" tornando-a linear. Não faremos essa etapa pois nosso objetivo é rodar um algoritmo não linear em seguida.

In [ ]:
# aplicação do get_dummies do pandas para dummização de variáveis categóricas
df = pd.get_dummies(df, drop_first=True)

In [ ]:
# verificação do tamamno do DataSet
df.shape

Aumentamos das 26 colunas originais para mais de 15mil colunas!

# Passo 4 - Treinando o modelo de Machine Learning

Essa é a etapa mais simples mas que exige mais experiência.

1) qual algoritmos escolher?

Sabemos que nosso problema se trata de prever uma variável continua (uma nota) e como temos diversas notas para treina-lo podemos usar uma abordagem supervisionado.

(obs: poderiamos tentar transformar cada intervalo de notas em uma categoria, ex: [0,1] -> 1, [1,2] -> 2 .... [9,10] -> 10 e prever usando um algoritmo de classificação, mas como as notas fazem sentido como uma variável numérica, ou seja, a nota seguinte é a nota anterior + 1, um algoritmo de classificação dificilmente superaria um de regressão já que as probabilidades de cada classe prevista em uma classificação é independente, é como se uma nota não tivesse nenhuma relação com a outra e isso não é verdade no nosso caso).

Então:

    Supervisionado -> Regressão
    
    
Mas há dezenas de algoritmos de regressão (ver todos os disponiveis no sklearn nesse link: http://scikit-learn.org/stable/supervised_learning.html )

Alguns deles:
- MQO (o mais simples que estudamos em estatistica e abrimos o código na aula2 de machine learning).
- Ridge -> Método de Shrinkage. Conrola sobreajuste com L1

    $L = ∑( Ŷi– Yi)^2 + λ∑ β^2$


- lasso -> Também método de Shrinkage. controle sobreajuste com L2

   $ L = ∑( Ŷi– Yi)^2 + λ∑ |β|$


- elasticnet -> combinação de Lasso + Ridge
- Regressão Bayesiana -> Retorna distribuições de probabilidade ao invés de simples valores.

    Ex: https://towardsdatascience.com/introduction-to-bayesian-linear-regression-e66e60791ea7?gi=392eaf9ea3b 
    
- Arvores aleatórias -> insere não lineariedades
- Redes neurais -> também insere não lineariedades.
- Ensambles -> Combinação de diferentes modelos.
- Etc


Qual usar ?

No nosso exemplo vamos usar o mais simples MQO para usar de comparativo e um extremamente poderoso que são as arvores aleatórias (que ainda não vimos seu funcionamento).

Na prática, quando colocamos um modelo em produção, podemos rodar diferentes algoritmos (que tem suas próprias vantagens e desvantagens) e pegar uma combinação desses diferentes algoritmos como o resultado final. A essa combinação de modelos damos o nome de **ensamble**.

Para rodar qualquer modelo de machine learning desses, vamos passar pelos mesmos passos:

    1) Importar o modelo desejado.
    2) Instanciar em uma variável com os parâmetros desejados (ainda não vimos os parâmetros).
    3) Separar nossos dados em variáveis explicativas (X) e explicadas/target (Y)
    4) Separar nossos dados em Treino e Teste
    5) Treinar o Modelo com o .fit()
    6) Analisar as métricas, se não estiver boas, voltamos aos passos anteriores de trabalhar com as variáveis.
    7) Estando tudo ok, rodamos previsões

## Importar modelos

In [ ]:
# importando os modelos
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

## Instanciar os modelos em variáveis

In [ ]:
# instanciando os modelos
modelo_MQO = LinearRegression()
modelo_RF = RandomForestRegressor()

## Separar os dados em X e Y

In [ ]:
# separando os dados em dados em X e Y
X = df.drop(['imdb_score'], axis = 1)
Y = df['imdb_score']

## Separar os dados em Treino e Teste 

Esse ponto é crucial em machine learning e falaremos mais vezes sobre diferentes técnicas em "splitar" os dados.

In [ ]:
# importando a biblioteca de split de dados do Sklearn
from sklearn.model_selection import train_test_split
# separando os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

In [ ]:
# imprimindo o tamanho dos DataSet
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Treinar o Modelo 

Para que o algoritmo descrubra os melhores betas (que produzem os menores erros) - ou os melhores otimizadores

In [ ]:
# treinando o modelo linear
modelo_MQO.fit(X_train, y_train)

## Calculando e analisando as métricas

Após treinar o modelo podemos analisar suas métricas e aqui é uma parte sensivel, vamos entender algumas métricas.**

In [ ]:
# importando as bibliotecas
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
# previsao dos dados de treino para calcular as métricas
yhat_train = modelo_MQO.predict(X_train) 
# previsao dos dados de teste para calcular métricas
yhat_test = modelo_MQO.predict(X_test) 

# imprimindo as métricas de treino
print('-----Dados de Treino-----')
print('MSE - treino', mean_squared_error(y_train, yhat_train))
print('MAE - treino', median_absolute_error(y_train, yhat_train))
print('R2 - treino', r2_score(y_train, yhat_train))
# imprimindo as métricas de teste
print('\n-----Dados de Teste-----')
print('MSE - test', mean_squared_error(y_test, yhat_test))
print('MAE - test', median_absolute_error(y_test, yhat_test))
print('R2 - test', r2_score(y_test, yhat_test))

Vamos entender primeiro as métricas e em seguida entender a diferença entre os números de treino e de teste.

**O que é o MSE e o MAE**
MSE: Erros médios quadraticos.
MAE: Erros médios absolutos.

Qual a diferença entre eles?

Pensemos no caso de um dataset com muitos outlinears, qual das duas métricas será mais prejudica (ou seja, qual métrica é mais sensivel a dados muito fora do padrão?). Nosso métrica ao quadrado é mais sensivel pois pegamos a diferença entre o valor predito e o realizado e elevamos ao quadrado tornando a diferença ainda maior. Já o MAE não é tão sensivel aos valores extremos pois consideramos apenas a distância absoluta que da o mesmo peso a distâncias grandes e pequenas.
Ex: Valor predito = 8, Valor Real = 5.
MAE= 8-5 = 3
MSE= (8-5)^2 = 9 -> Penaliza erros maiores.


Já o R2 é a mesma intepretação que demos em modelagem estatistica (% da variação de Y explicada por todas as variáveis explicativas X). Só que agora nosso objetivo é aumentar ao máximo nosso R2.



** Métricas de Treino e Métricas de Teste **

O grande objetivo de um modelo de aprendizado de maquina (ML) é termos um modelo que é preditivo para novos dados, ou seja, precisamos de um modelo que seja bastante genérico, que depois de treinado pode pegar dados que não estavam na amostra e consiga prever seu target com uma boa precisão.

Então ter boas métricas nos dados de treino não significam necessariamente que o modelo é um bom preditor. Para isso ele precisa ter boas métricas nos dados de teste! (aquela parte dos dados que separamos e não participaram do treino). Se nosso algoritmo é capaz de prever de maneira satisfatória nossos dados de teste (que são dados novos para ele já que não participaram do treino) esse é um bom modelo.

Quando as métricas dos dados de treino estão muito superiores aos dados de teste (como no nosso exemplo) significa que estamos fazendo um superajuste/sobreajuste ou como gostamos de chamar o modelo está com overfiting. Significa basicamente que nosso modelo se tornou especialista em prever nossos dados de treino mas não necessariamente é um bom modelo para prever dados fora do treinamento. Há muitas formas de contornar isso que veremos nas próximas aulas, mas a mais importante que faremos aqui é tentar equilibrar a complexidade do modelo, basicamente colocamos mais variáveis do que seria necessario.

É aqui que voltamos ao passo de escolher e tratar variaveis e rodamos tudo novamente.
Vamos fazer todos os processos na próxima linha e colocar um modelo um pouco mais simples:


<img src="img\overfit.png" style="height:250px">

## Melhorando o ajuste do modelo

Vamos remover algumas colunas que podem estar causando um superajuste/overfiting no nosso modelo

In [ ]:
# listando as colunas do DataFrame
list(df_raw)

## Estratégia de eliminação de colunas

Para o DataSet do IMDB, vamos manter as colunas com likes no título pois representam uma característica importante dos filmens, e remover as colunas com nomes, links, unnamed e keywords pois são muito específicas e não diferenciam os filmas entre si.

In [ ]:
# primeiro vamos identificar as colunas
# função para manutenção e eliminação de colunas
for column in list(df_raw):
    if 'likes' in column:
        pass
    elif 'name' in column or 'link' in column or 'Unnamed: 0' in column or 'plot_keywords' in column:
        print(column)

In [ ]:
# vamos eliminar as colunas

# definindo uma lista para as colunas
overfiting_columns = []
# aplicação da função para escolher as colunas a serem eliminadas
for column in list(df):
    if 'likes' in column:
        pass
    elif 'name' in column or 'link' in column or 'Unnamed: 0' in column:
        overfiting_columns.append(column)

# imprimindo as colunas a serem eliminadas
print(f'#columns_to_drop: {len(overfiting_columns)}')
# dropando as colunas selecionadas no DataFrame original
df = df.drop(overfiting_columns, axis=1)
# criando as variáveis Dummies
df = pd.get_dummies(df)

# imprimindo o novo tamanho do DataFrame
print(f'new_shape: {df.shape}')

## Rodando novamente uma Regressão Linear

Vamos verificar como os dados respondem a uma nova regressão linear considerando o DataFrame sem as colunas removidas para eliminação do Overfitting

In [ ]:
# rodando uma regressão linear através de uma função
def run_mqo(df, model):
    # separando os dados em X e Y
    X = df.drop(['imdb_score'], axis = 1)
    Y = df['imdb_score']
    # split dos dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25) 
    # instanciando o modelo
    model.fit(X_train,y_train)
    # fazendo previsões com os dados de treino 
    yhat_train = model.predict(X_train) 
    # fazendo previsões com os dados de teste
    yhat_test = model.predict(X_test[list(X_train)])
    
    # imprimindo as métricas para os dados de treino
    print('-----Dados de Treino-----')
    print('MSE - treino', mean_squared_error(y_train, yhat_train))
    print('MAE - treino', median_absolute_error(y_train, yhat_train))
    print('R2 - treino', r2_score(y_train, yhat_train))
    
    # imprimindo as métricas para os dados de teste
    print('\n-----Dados de Teste-----')
    print('MSE - test', mean_squared_error(y_test, yhat_test))
    print('MAE - test', median_absolute_error(y_test, yhat_test))
    print('R2 - test', r2_score(y_test, yhat_test))

# rodando a função criada para rodar uma regressão linear
run_mqo(df, LinearRegression())

## Rodando novamente uma Random Forrest

Vamos passar agora agora um segundo modelo que consegue capturar, entre outras coisas, não lineariedades, uma vez que o modelo linear não está respondendo.

In [ ]:
# utilizando a mesma função para apliação de uma Random Forrest
run_mqo(df, RandomForestRegressor())

Parece bem melhor agora, tanto o R2 aumentou como ficou mais parecido entre treino e teste sugerindo que não estou dando overfiting no modelo.

Podemos ainda criar modelos um pouco mais complexos inserindo formas quadraticas, cubicas, etc para tentar capturar algum efeito não linear nos dados.

Idealmente plotariamos as variáveis e escolheriamos as que fazem mais sentido colocar variaveis quadraticas, etc. Mas na prática rodamos um algoritmo para passar um polinomio em todas as nossas variáveis como no código abaixo:

Obs: Colocamos todos os passos da regressão em uma função para não termos que ficar repetindo mais códigos.

## Definindo o melhor modelo para as utilização nas predições finais

In [ ]:
# criando uma função para imprimir as métricas do modelo
def print_metrics(y_train, yhat_train,y_test, yhat_test):
    # imprimindo as métricas de treino
    print('\n-----Dados de Treino-----')
    print('MSE - treino', mean_squared_error(y_train, yhat_train))
    print('MAE - treino', median_absolute_error(y_train, yhat_train))
    print('R2 - treino', r2_score(y_train, yhat_train))
    # imprimindo as métricas de teste
    print('\n-----Dados de Teste-----')
    print('MSE - test', mean_squared_error(y_test, yhat_test))
    print('MAE - test', median_absolute_error(y_test, yhat_test))
    print('R2 - test', r2_score(y_test, yhat_test))

In [ ]:
# importando as bibliotecas
from sklearn.preprocessing import PolynomialFeatures

# definindo uma variável com as colunas numéricas
numeric_columns = ['num_critic_for_reviews',
                     'duration',
                     'director_facebook_likes',
                     'actor_3_facebook_likes',
                     'actor_1_facebook_likes',
                     'num_voted_users',
                     'cast_total_facebook_likes',
                     'facenumber_in_poster',
                     'num_user_for_reviews',
                     'budget',
                     'title_year',
                     'actor_2_facebook_likes',
                     'aspect_ratio',
                     'movie_facebook_likes']

# criando uma lista vazia para variáveis categóricas
dummies = []

# colocando as vairáveis categóricas em uma lista
for column in list(df):
    if 'imdb_score' in column:
        pass
    elif column not in numeric_columns:
        dummies.append(column)

# criando uma função para modelagem com escolha do grau de transformação polinomial 
def run_model_with_poly(dataframe, poly_n, modelo):
    # definição do grau da função polinomial de acordo com o parâmetro inicial   
    poly = PolynomialFeatures(degree=poly_n)
    # definição dos dados de entrada X e imprimindo o tamanho da variável
    newX = dataframe[dummies+numeric_columns]
    print('newX shape', newX.shape)
    # definição da variável alvo 
    newY = dataframe['imdb_score']
    # aplicação da transformação polinomial nos dados de entrada X e imprimindo o tamanho
    df_temp = pd.DataFrame(poly.fit_transform(newX[numeric_columns]))
    print('\ndf_temp shape', df_temp.shape)
    # concatenando os dados numericos polinomiais com as variávis dummies
    df_temp = pd.concat([df_temp,dataframe[dummies].reset_index()], axis=1)
    print('df_temp shape', df_temp.shape)
    # splitando os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(df_temp,newY,test_size=0.2)
    # imprimindo o tamanho dos dados de entrada X de treino e teste
    print('\ntrain shape', X_train.shape)
    print('teste shape', X_test.shape)
    # ajustando o modelo
    modelo.fit(X_train, y_train)
    # fazendo predições para os dados de treino
    yhat = modelo.predict(X_train)
    # fazendo predições para os dados de teste
    yhat_test = modelo.predict(X_test[list(X_train)])
    # imprimindo as métricas e retornando o modelo
    print_metrics(y_train, yhat, y_test, yhat_test)
    return modelo

Vamos testar os modelos de Regressão Linear e Random Forrest com diferentes graus de funções polinomiais para saber qual é a melhor combinação para ser adotada em nossas predições.

In [ ]:
# rodando a função para modelagem com o modelo linear
# parametriando a função polinomial com grau 1
model1 = run_model_with_poly(df, 1, LinearRegression())

In [ ]:
# rodando a função para modelagem com o modelo linear
# parametriando a função polinomial com grau 2
model2 = run_model_with_poly(df, 2, LinearRegression())

In [ ]:
# rodando a função para modelagem com o modelo Random Forrest
# parametriando a função polinomial com grau 1
model3 = run_model_with_poly(df, 1, RandomForestRegressor())

In [ ]:
# rodando a função para modelagem com o modelo random forrest
# parametriando a função polinomial com grau 2
model4 = run_model_with_poly(df, 2, RandomForestRegressor())

In [ ]:
# rodando a função para modelagem com o modelo random forrest
# parametriando a função polinomial com grau 2
model5 = run_model_with_poly(df, 3, RandomForestRegressor())

Comparando nossos dois modelos (MQO vs Arvores Aleatórias) parece que as segunda consegue superar as métricas no treino e no teste então vamos usar esse modelo em produção (ou seja, para realizar nosso trabalho).

Estamos buscando uma convicção de que o modelo treinado tem uma boa relação entre o viés e variância com a parametrização pela complexidade do próprio modelo. 

Novamente, quanto mais complexo mais se consegue capturar efeitos não previsto pelo modelo, como comportamentos não lineares e interação das variáveis.

Porém, tornando o modelo menos genérico, ou seja, diminuindo os scores dos dados de teste, devemos treinar o modelo final com os parâmetros que conseguiram os melhores resultados nos dados de teste.

## Escolhendo o melhor modelo

Dos nossos testes parece que a melhor combinação foi um RandomForestRegressor com polinomios = 1. Então vamos fazer o treinamento do modelo final com TODOS os dados (agora não podemos mais confiar nas métrica pois não teremos dados de teste)

In [ ]:
# definindo os dados
Y = df['imdb_score']
X = df.drop(['imdb_score'], axis=1)
# rodando o melhor modelo
final_model = RandomForestRegressor().fit(X,Y)

# Passo 5 - Fazer Predições

Depois que temos o melhor modelo treinado vamos pegar os dados do mundo real e fazer predições.

In [ ]:
# lendo os dados originais, criando uma cópia e verificando o DataFrame
data_raw = pd.read_csv('../../99 Datasets/imdb_test.zip')
data = data_raw.copy(deep=True)
data.head()


Nesse nosso caso, temos o target nos dados "reais", geralmente não o teriamos. Aqui poderemos saber a perfomance do nosso algoritmo.

## Fazendo os mesmos tratamento nos dados 

Todos os tratamentos de dados que fizemos no dataset original precisamos refazer aqui.

In [ ]:
# dropando as colunas que causam overfit
data = data.drop(columns_to_drop, axis=1)
# preenchendo as colunas numéricas com a média
data[list(df_numeric)] = data[list(df_numeric)].fillna(data.mean())
# preenchendo os campos nulos
data = data.fillna('na')
# criando DUMMIES para as colunas categóricas
data = pd.get_dummies(data)
# separando os dados da variável alvo no dataset Y
Ydata = data['imdb_score']

In [ ]:
# verificando o número de colunas de X
len(list(X))

In [ ]:
# verificando o numero de colunas do DataFrame
len(list(data))

In [ ]:
# função para zerar as DUMMIES que não estão em X
for i, dummy in enumerate(list(X)):
    if dummy not in list(data):
        data[dummy] = 0

print(f'dummies adicionadas:{i}')

In [ ]:
# atribuindo todas as DUMMIES aos dados de entrada X
Xdata = data[list(X)]

## Fazendo as predições

In [ ]:
# fazendo as predições com o melhor modelo já treinado
yhat = final_model.predict(Xdata)

In [ ]:
# Perfeito! Temos um array com os valores previstos
yhat

- Como podemos saber de qual filme se trata cada um dos score previstos?


- o algoritmo não diz qual é o nome do filme, mas a ordem em que passamos os dados se mantém ...


- então podemos adicionar uma coluna no nosso dataframe original com as predições!


# Respondendo a pergunta do nosso Desafio:

In [ ]:
# adicionando os valores previstos ao DataFrame original
data_raw['score_previsto'] = yhat
data_raw.head()

In [ ]:
# criando um novo DataFrame com o Score Previsto, título do filme e o orçamento
result = data_raw[['movie_title','budget','score_previsto']]
# criando uma nova coluna RATIO com o orçamento (em USD Mi) dividido pelo Score Previsto
result['ratio'] = (result['budget']/1000000)/(result['score_previsto'])

Para a solução, devemos pensar que os filmens com menos Bugdet irão custar mais barato, e os filmes com maior Score Previsto são aqueles que tem melhor apreciação do público, portanto o RATIO tem a idéia de capturar as oportunidades mais baratas e com melhor avaliação.

In [ ]:
# "piores" 10 filmes nas regras desse desafio, são os filmes que os scores custam mais caro
result.sort_values(by=['ratio'], ascending=False).head(10)

In [ ]:
# "melhores" 10 filmes nas regras desse desafio são aqueles com o menos ratio
result.sort_values(by=['ratio'], ascending=True).head(10)

# Próximo passo, colocar o modelo em produção ...